In [80]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import sklearn.linear_model as skl_lm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf


import plotly.express as px

%matplotlib inline
plt.style.use('seaborn-white')

In [56]:
PROJECT_ROOT_DIR = "/Users/daniel421/Desktop/STAT_724/ds_724/Final_Project"
# CHAPTER_ID = "Lin_regression"
# IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
# os.makedirs(IMAGES_PATH, exist_ok=True)

#### Column indices to potentially drop: 4,5,6 and -1

In [69]:
import os
#UP_DIR = '/Users/daniel421/Desktop/STAT_724/ISLR_data'
csv_file = os.path.join(PROJECT_ROOT_DIR,'listings_6.csv')
abnb = pd.read_csv(csv_file, low_memory=False)
abnb = pd.DataFrame(data = abnb)
abnb = abnb.drop(abnb.columns[[4,5,6,-1]], axis = 1)

In [70]:
len(abnb)

15991

Sklearn SimpleImputer

In [71]:
abnb.fillna(abnb.mean(), inplace=True)

In [54]:
# s_imputer = SimpleImputer(missing_values= np.nan, strategy = 'mean', verbose=0)
# s_imputer = s_imputer.fit(abnb[1:,1:])
# abnb = s_imputer.transform(abnb)

TypeError: '(slice(1, None, None), slice(1, None, None))' is an invalid key

In [72]:
abnb = pd.DataFrame(data = abnb)
print(abnb.columns)

Index(['host_response_rate', 'host_acceptance_rate', 'host_listings_count',
       'host_total_listings_count', 'accommodates', 'bedrooms', 'beds',
       'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'host_is_superhost2', '

In [24]:
# abnb.dtypes

In [73]:
# abnb['host_response_rate'] = abnb['host_response_rate'].astype(float, errors = 'raise')
# abnb['host_response_rate'] = pd.to_numeric(abnb['host_response_rate'], errors='coerce')
# abnb['host_response_rate'] = (abnb['host_response_rate'].str.strip('%').astype(float))
# abnb['host_response_rate'] = abnb['host_response_rate']/100
# abnb['host_acceptance_rate'] = (abnb['host_acceptance_rate'].str.strip('%').astype(float))
# abnb['host_acceptance_rate'] = abnb['host_acceptance_rate']/100


## Need to scale the dataframe?

#### Column indices to potentially drop: 4,5,6 and -1

In [74]:
# rescaling the variables (both)
abnb_columns = abnb.columns
scaler = MinMaxScaler()
abnb = scaler.fit_transform(abnb)

# rename columns (since now its an np array)
abnb = pd.DataFrame(abnb)
abnb.columns = abnb_columns

abnb.head()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,...,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost2,host_identity_verified2,has_availability2,instant_bookable2,dmy_Hotel room,dmy_Private room,dmy_Shared room
0,0.35,0.18,0.002549,0.002549,0.0625,0.047935,0.000000,0.015002,0.029029,0.562281,...,0.000000,0.0,0.002353,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.21,0.76,0.000319,0.000319,0.1875,0.000000,0.117647,0.007301,0.000000,0.364682,...,0.000000,0.0,0.036032,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.00,1.00,0.000319,0.000319,0.1250,0.000000,0.000000,0.006001,0.029029,0.364682,...,0.006079,0.0,0.003824,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,1.00,0.25,0.000319,0.000319,0.2500,0.125000,0.058824,0.027503,0.004004,0.562281,...,0.000000,0.0,0.000074,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,1.00,1.00,0.000319,0.000319,0.1250,0.000000,0.000000,0.006801,0.001001,0.006503,...,0.003040,0.0,0.027134,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Split into train and test

creating y_train and y_test, X_train, X_test

In [75]:
abnb_train, abnb_test = train_test_split(abnb, train_size = 0.7, test_size = 0.3, random_state = 10)

y_train = abnb_train['price']
y_test = abnb_test['price']

X_train = abnb_train.drop('price', axis = 1)
X_test = abnb_test.drop('price', axis = 1)

In [30]:
# print(len(X_test))
# print(len(y_test))

11193

In [76]:
print(len(X_train))
print(len(y_train))

11193
11193


In [77]:
isinstance(abnb_train, pd.DataFrame)

True

## Building a model without cross-validation

In [78]:
print(len(X_train.columns))
print(X_train.shape, y_train.shape)

41
(11193, 41) (11193,)


In [81]:
# first model with an arbitrary choice of n_features
# running RFE with number of features = 20

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=20)             
rfe = rfe.fit(X_train, y_train)

## Begin by implementing K-fold cross validation